<a href="https://colab.research.google.com/github/AlexeyTri/MyMachineLearningSeminars/blob/main/seminar21/%D0%9E%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5_P%D0%B0%D0%BD%D0%B6%D0%B8%D1%80%D0%BE%D0%B2%D0%BD%D0%B8%D1%8E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!!pip install --quiet -U  pyserini jsonlines scikit-surprise fastrank

In [ ]:
!pip install  faiss

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# DATA_PATH = './new_data'
# !mkdir new_data
# !cp /content/drive/MyDrive/'МФТИ машинное обучение'/'Домашки'/'2-й семестр'/'dataset for seminar 19'/lenta-ru-filtred.csv new_data
# !cp /content/drive/MyDrive/'МФТИ машинное обучение'/'Домашки'/'2-й семестр'/'dataset for seminar 19'/MSLR-WEB10K.zip new_data/
!unzip new_data/MSLR-WEB10K.zip -d new_data/mslr

Библиотеки

In [36]:
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib import gridspec

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import scipy
import json
import re
import os
import jsonlines
import lightgbm as lgb

from surprise import Dataset, Reader, SVD
from fastrank import CDataset, TrainRequest
from sklearn.datasets import load_svmlight_file
# строка кода не работает
# from pyserini.search import SimpleSearcher

# Learning to Rank

Загружаем DataSet

In [29]:
DATA_PATH = 'new_data/mslr/Fold1'
(X_train, y_train, qid_train) = load_svmlight_file(
    os.path.join(DATA_PATH, 'train.txt'),
    dtype=np.float32,
    zero_based=False,
    query_id=True)
(X_val, y_val, qid_val) = load_svmlight_file(
    os.path.join(DATA_PATH, 'vali.txt'),
    dtype=np.float32,
    zero_based=False,
    query_id=True)
(X_test, y_test, qid_test) = load_svmlight_file(
    os.path.join(DATA_PATH, 'test.txt'),
    dtype=np.float32,
    zero_based=False,
    query_id=True)

train_size = 50000
X_train = X_train.toarray()[:train_size]
y_train = y_train[:train_size]
qid_train = qid_train[:train_size]

val_size = 10000
X_val = X_val.toarray()[:val_size]
y_val = y_val[:val_size]
qid_val = qid_val[:val_size]

test_size = 1000
X_test = X_test.toarray()[:test_size]
y_test = y_test[:test_size]
qid_test = qid_test[:test_size]

In [30]:
ids = np.random.RandomState(42).randint(0, len(X_train), size=5)
(X_train[ids][:,:6], y_train[ids], qid_train[ids])

(array([[2.      , 0.      , 1.      , 1.      , 2.      , 1.      ],
        [3.      , 0.      , 3.      , 2.      , 3.      , 1.      ],
        [2.      , 1.      , 2.      , 0.      , 2.      , 1.      ],
        [2.      , 0.      , 1.      , 0.      , 2.      , 1.      ],
        [6.      , 2.      , 1.      , 1.      , 6.      , 0.857143]],
       dtype=float32),
 array([0., 0., 2., 0., 1.]),
 array([2071,  151, 4966, 5971, 1486]))

Определяем модель

In [31]:
train_request = TrainRequest.random_forest()
params = train_request.params
params.num_trees = 100
params.feature_sampling_rate = 0.5
params.instance_sampling_rate = 0.5

params.quit = True
params.seed =42

Специальный формат данных

In [32]:
dataset_train = CDataset.from_numpy(X_train, y_train, qid_train)
dataset_test = CDataset.from_numpy(X_test, y_test, qid_test)

features = dataset_train.feature_names()
train_queries = sorted(dataset_train.queries())
test_queries = sorted(dataset_test.queries())

train = dataset_train.subsample_queries(train_queries)
test = dataset_test.subsample_queries(test_queries)

In [33]:
model = train.train_model(train_request)

\begin{align}
        \mathbf{DCG@K} &= \sum_{k=1}^K \frac{r_{true}(π^{-1}(k))}{log_2(k+1)}, &  \mathbf{IDCG@K}&= \sum_{k=1}^K \frac{1}{log_2(k+1)}, & \mathbf{nDCG@K} &= \frac{\mathbf{DCG@K}}{\mathbf{IDCG@K}}
    \end{align}

In [ ]:
test.evaluate(model, 'NDCG@5')

# Рекомендательная система

In [ ]:
data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()
dataframe = pd.DataFrame(
    trainset.all_ratings(),
    columns=['id', 'iid', 'raiting'])
dataframe.sample(5, random_state=42)

In [39]:
algo = SVD()
algo = algo.fit(trainset)

In [41]:
(algo.predict('10','4').est,
 algo.predict('10','5').est,
 algo.predict('10','6').est)

(4.007853728232699, 3.590404992491329, 4.1198711505764)

# Поисковая система

In [42]:
data = pd.read_csv('/content/new_data/lenta-ru-filtred.csv')

In [ ]:
data

In [44]:
from pandas.core.algorithms import doc
COLLECTION_DIR = 'new_data/collection/lenta'
if not os.path.exists(COLLECTION_DIR):
    os.makedirs(COLLECTION_DIR)

collection = data.values
batch_size = 32768
for ind in tqdm(range(0, len(collection), batch_size)):
    files=[{'id': ind+1,
            'contents': doc[0],
            'tag': doc[1],
            'date': doc[-1]}
           for i, doc in  enumerate(collection[ind:ind+batch_size])]
    with jsonlines.open(os.path.join(COLLECTION_DIR, '{}.jsonl'.format(ind)),
                        mode='w') as writer:
                        writer.write_all(files)

  0%|          | 0/25 [00:00<?, ?it/s]

In [45]:
with open(os.path.join(COLLECTION_DIR, 
                       '{}.jsonl'.format(ind))) as f:
  print('\n'.join(f.read().splitlines()[-5:]))

{"id": 786433, "contents": "Певец Сергей Шнуров раскритиковал свою коллегу Полину Гагарину в эфире шоу «Голос» на Первом канале. Момент передачи опубликован на YouTube. После выступления подопечных Гагариной Шнуров обратил внимание на номер 17-летней участницы Аланы Чочиевой. По мнению певца, исполненная девушкой песня Аллы Пугачевой «Опять метель» была выбрана неудачно. «Перед нами ангел, который поет: \"Сильная женщина плачет у окна\". Откуда это, зачем?» — возмутился он. Однако другой наставник, продюсер Константин Меладзе, с ним не согласился. По его мнению, Чочиевой досталась сложная композиция, и она спела ее по-своему хорошо. Гагарина также поддержала свою подопечную, отметив, что та «не выступает с табличкой», на которой указан ее возраст. Это не первый конфликтный эпизод на «Голосе». До этого Гагарину оскорбило поведение уральского певца Арсения Курчанина. Во время выступления артист насыпал на стол певицы семечки, однако позже их убрал Шнуров.", "tag": "ТВ и радио", "date": 9

Построение индекса

In [46]:
INDEX_DIR = 'new_data/index/lenta'
if not os.path.exists(INDEX_DIR):
    os.makedirs(INDEX_DIR)

In [ ]:
!python -m pyserini.index -collection JsonCollection -generator DefaultLuceneDocumentGenerator \
 -threads 16 -input {COLLECTION_DIR} \
 -index {INDEX_DIR} -storePositions -storeDocvectors -storeRaw

## Испльзуем поисковый индекс


* BM-25

Пусть дан запрос Q, содержащий слова q_1,...,q_n тогда функция BM25 дает следующую оценку релеваности документа D запросу Q

\begin{align}
        \mathbf{score(D,Q)} &= \sum_{i=1}^n 
        \mathbf{IDF(q_i)}) \cdot \frac{tf(q_i, D) \cdot (k_1 + 1)}{tf(q_i, D) +  k_1 \cdot (1 - b + b \cdot \frac{|D|}{avgdl})}
    \end{align}

где tf(q_i, D) есть частота слова q_i в документе D, |D| есть длина документа, а avgdl- средняя длина документа в коллекции. k1 и b - свободные коэффициенты

In [ ]:
# код не работает

# searcher = SimpleSearcher(INDEX_DIR)
# searcher.set_bm25()
# hits = searcher.search('Поисковик', k=1)

# for i in range(len(hits)):
#     doc = searcher.doc(hits[i].docid)
#     print(doc.raw())

* QLD

Идея: латентное размещение Дирихле (отсылка к тематическим моделям)

In [ ]:
# код не работает

# searcher = SimpleSearcher(INDEX_DIR)
# searcher.set_qld()
# hits = searcher.search('Поисковик', k=1)

# for i in range(len(hits)):
#     doc = searcher.doc(hits[i].docid)
#     print(doc.raw())

* RM3

Пусть дан запрос Q, содержащий слова q_1,...,q_n тогда функция BM25 дает следующую оценку релеваности токена t запросу Q

\begin{align}
        \mathbf{score(t,Q)} &= \alpha \cdot softmax \left( \sum_{d\in{PRD}}^{} \left[\frac{tf(t, D)}{|d|} \cdot \prod_{i=1}^k \frac{tf(q_i, d)+μp(q_i)}{|d|+μ} \right]\right) + (1-α) \cdot \left(\frac{tf(t, Q)}{|Q|}\right)
    \end{align}


где tf(t, D) есть частота токена t в документе d, |d| есть длина документа, а avgdl- средняя длина документа в коллекции. a и m - свободные коэффициенты

In [ ]:
# код не работает

# searcher = SimpleSearcher(INDEX_DIR)
# searcher.set_rm3()
# hits = searcher.search('Поисковик', k=1)

# for i in range(len(hits)):
#     doc = searcher.doc(hits[i].docid)
#     print(doc.raw())